In [1]:
!pip install gymnasium

In [2]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [3]:
values_time = []
policy_time = []

In [4]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [5]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [6]:
env.observation_space.n

16

In [7]:
env.action_space.n

4

In [8]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [9]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 70)

In [10]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 20)

In [11]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 9)

In [12]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 2)

In [13]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 79)

In [14]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [15]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 57/1000
Average number of steps: 12.192982456140351


In [17]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 115/1000
Average number of steps: 15.6


In [18]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 769/1000
Average number of steps: 43.19115734720416


In [19]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [20]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [22]:
np.all(v_values_1 >= v_values_0)

True

In [23]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [24]:
np.all(v_values_2 >= v_values_1)

True

In [25]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [26]:
np.all(v_values_3 >= v_values_2)

True

In [27]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [28]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [29]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [30]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [31]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [32]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [33]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 47)

In [34]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 790/1000
Average number of steps: 43.540506329113924


In [35]:
def policy_iteration(env, max_iters=500, gamma=0.9):
  # initialize
  pi_values = np.zeros(env.observation_space.n)
  #pi_values = np.random.choice(env.action_space.n, size=(env.observation_space.n))

  for i in range(max_iters):
    prev_pi_values = np.copy(pi_values)

    # policy evaluation
    v_values = policy_evaluation(env, pi_values)

    # policy improvement
    pi_values = policy_extraction(env, v_values, gamma)

    #if np.all(np.isclose(pi_values, prev_pi_values)):
    #  print(f'Converged at {i}-th iteration.')
    #  break

    if np.all(pi_values == prev_pi_values):
      print(f'Converged at {i}-th iteration.')
      break

  return pi_values

# FrozenLake-v1

In [36]:
env = gym.make('FrozenLake-v1')

In [37]:
# Value Iteration
start = time.time()
optimal_v_values = value_iteration(env)
optimal_pi_values = policy_extraction(env, optimal_v_values)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

values_time.append(end - start)

Converged at 79-th iteration.
Number of successes: 771/1000
Average number of steps: 43.64980544747082


In [38]:
# Policy Iteration
start = time.time()
optimal_pi_values = policy_iteration(env)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

policy_time.append(end - start)

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 5-th iteration.
Number of successes: 787/1000
Average number of steps: 44.23125794155019


# FrozenLake8x8-v1

In [39]:
env = gym.make('FrozenLake8x8-v1')

In [40]:
# Value Iteration
start = time.time()
optimal_v_values = value_iteration(env)
optimal_pi_values = policy_extraction(env, optimal_v_values)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

values_time.append(end - start)

Converged at 117-th iteration.
Number of successes: 747/1000
Average number of steps: 75.16867469879519


In [41]:
# Policy Iteration
start = time.time()
optimal_pi_values = policy_iteration(env)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

policy_time.append(end - start)

Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 9-th iteration.
Number of successes: 758/1000
Average number of steps: 75.2928759894459


# Taxi-v3

In [42]:
env = gym.make('Taxi-v3')

In [43]:
# Value Iteration
start = time.time()
optimal_v_values = value_iteration(env)
optimal_pi_values = policy_extraction(env, optimal_v_values)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

values_time.append(end - start)

Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.193


In [44]:
# Policy Iteration
start = time.time()
optimal_pi_values = policy_iteration(env)
end = time.time()

# Test Policy
play_multiple_times(env, optimal_pi_values, 1000)

policy_time.append(end - start)

Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 16-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.011


# Nhận xét

In [45]:
# Thời gian thực thi của thuật toán Value Iteration
values_time

[0.1287217140197754, 0.7260980606079102, 5.884201526641846]

In [46]:
# Thời gian thực thi của thuật toán Policy Iteration
policy_time

[0.10814142227172852, 1.0225796699523926, 14.559151887893677]

In [47]:
np.mean(values_time)

2.2463404337565103

In [48]:
np.mean(policy_time)

5.229957660039266

In [49]:
np.mean(policy_time) / np.mean(values_time)

2.3282124033592324

In [52]:
# Trong các toy games ở đây thì Value Iteration cho ra kết quả nhanh hơn (trung bình thời gian chạy) khoảng 2.33 lần
# Policy Iteration thường hội tụ nhanh hơn, nhưng tính toán phức tạp hơn
#         Value Iteration --- Policy Iteration
# FrozenLake-v1:       79 --- 5
# FrozenLake8x8-v1:   117 --- 9
# Taxi-v3:            116 --- 16
# Mặc dù hội tụ nhanh hơn, nhưng Policy Iteration còn phải thực hiện bước policy_extraction trên nhiều iters, nên sẽ mất nhiều thời gian thực thi hơn